In [5]:
import numpy as np
import pandas as pd

In [43]:
df_all = pd.read_csv('./tmp/df_test.csv')
df_all = df_all.set_index('case_name')

In [44]:
df_all

,aircon,ventilation,exhaust_a,exhaust_b,exhaust_off,office,countTimeMean_onlyFloating
case_name,,,,,,,
office1_0_164_bout,-1.387842,0.282251,-0.868459,1.151465,-0.403652,office1,3271.053000
office1_960_123_bout,1.293562,-0.204658,-0.868459,1.151465,-0.403652,office1,1546.898667
office1_960_164_aout,1.293562,0.282251,1.151465,-0.868459,-0.403652,office1,999.284000
office2_0_164_bout,-1.387842,0.282251,-0.868459,1.151465,-0.403652,office2,2322.241333
office2_960_164_aout,1.293562,0.282251,1.151465,-0.868459,-0.403652,office2,2461.452000
...,...,...,...,...,...,...,...
office33_960_164_aout,1.293562,0.282251,1.151465,-0.868459,-0.403652,office33,1761.043333
office33_660_0,0.455624,-1.665384,-0.868459,-0.868459,2.477379,office33,1985.264333
scan1_0_164_bout,-1.387842,0.282251,-0.868459,1.151465,-0.403652,scan1,1362.842000


点群

In [45]:
point_cloud_dict = np.load('./saved_obj/PointCloud_dict_test.npy', allow_pickle=True).item()

In [46]:
def standardization(x:np.array, axis=None, ddof=0):
    x_mean = x.mean(axis=axis, keepdims=True)
    x_std = x.std(axis=axis, keepdims=True, ddof=ddof)
    return (x - x_mean) / x_std

In [47]:
# officeデータのみ井田さん式で標準化

# testデータ
pc_list = []
for office in df_all['office']:
    if office=='scan1':
        continue
    pc_list.append(point_cloud_dict[office])

pc_array = np.array(pc_list)

a = pc_array.reshape(-1,3)
a_standardized = standardization(a, axis=0)
pc_array_office_test = a_standardized.reshape(pc_array.shape)

In [48]:
point_cloud_list = []
for i in range(33):
    if i==29:
        continue
    point_cloud_list.append(point_cloud_dict[f'office{i+1}'])

point_cloud_array = np.array(point_cloud_list)

point_cloud_array.shape

(32, 2048, 3)

In [49]:
point_cloud_array_reshape = point_cloud_array.reshape(-1,3)
point_cloud_array_reshape.shape

(65536, 3)

In [50]:
x_mean = []
x_std = []

for i in range(3):
    x_mean.append(np.mean(point_cloud_array_reshape[:,i]))
    x_std.append(np.std(point_cloud_array_reshape[:,i], ddof=0))

print(x_mean)
print(x_std)

[4.060322, 2.6875114, 1.2308416]
[1.9615167, 1.2216018, 0.92776775]


In [51]:
# officeモデルのみで標準化定義

def standardization_by_office(array:np.array, x_mean, x_std):
    array_std = array.copy()
    for i in range(3):
        array_std[:,i] = (array[:,i] - x_mean[i]) / x_std[i]
    return array_std

In [52]:
# 読み込む場合
scan1_points = np.load('./saved_obj/scan1_points.npy')

In [53]:
scan1_points_std = standardization_by_office(scan1_points, x_mean, x_std)

In [54]:
pc_array_test = pc_array_office_test.copy()
for i in range(3):
    pc_array_test = np.concatenate((pc_array_test, np.expand_dims(scan1_points_std, axis=0)))

In [55]:
pc_array_test.shape

(99, 2048, 3)

データ整理

In [59]:
meta_all = df_all[['aircon', 'ventilation', 'exhaust_a', 'exhaust_b', 'exhaust_off']]
meta_scan = df_all[df_all['office']=='scan1'][['aircon', 'ventilation', 'exhaust_a', 'exhaust_b', 'exhaust_off']]
y_all = df_all['countTimeMean_onlyFloating']
y_scan = df_all[df_all['office']=='scan1']['countTimeMean_onlyFloating']
pc_all = pc_array_test
pc_scan = pc_array_test[-3:]

In [1]:
import keras

In [2]:
model_str = open('./saved_obj/best_model.json').read()
model = keras.models.model_from_json(model_str)
model.load_weights('./saved_obj/concat_scan_model/plus_scan1_weights_202307041515.hdf5')

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.006781976583074642), # learning_rateにoptunaの探索結果を直接代入
        loss='mse',
        metrics=['mae'],
        )

model.summary()

Model: "predictionnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 pointCloud (InputLayer)        [(None, 2048, 3)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2048, 32)     128         ['pointCloud[0][0]']             
                                                                                                  
 batch_normalization (BatchNorm  (None, 2048, 32)    128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 2048, 32)     0           ['batch_normalization

In [65]:
all_pred = model.predict([meta_all, pc_all])
scan_pred = model.predict([meta_scan, pc_scan])

1/1 [==============================] - 0s 23ms/step


In [31]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [68]:
scan_mae = mean_absolute_error(y_scan, scan_pred)
all_mae = mean_absolute_error(y_all, all_pred)
scan_rmse = np.sqrt(mean_squared_error(y_scan, scan_pred))
all_rmse = np.sqrt(mean_squared_error(y_all, all_pred))

In [69]:
df_result = pd.DataFrame([[scan_mae, scan_rmse], [all_mae, all_rmse]], columns=['MAE','RMSE'], index=['scan1','all'])
df_result

,MAE,RMSE
scan1,253.635605,283.469567
all,229.741842,296.957446
